In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import pickle
import re

In [ ]:
with open("/content/augmented_diverse_structure.pkl", "rb") as f:
    augmented_data = pickle.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Speech-NER/train.pkl", 'rb') as file:
    train = pickle.load(file)

with open("/content/drive/MyDrive/Speech-NER/test.pkl", 'rb') as file:
    test = pickle.load(file)

with open("/content/drive/MyDrive/Speech-NER/aug.pkl", 'rb') as file:
    aug = pickle.load(file)

In [ ]:
seed_data = [[], []]
for i in range(len(train[0])):
  if train[1][i] != []:
    seed_data[0].append(train[0][i])
    seed_data[1].append(train[1][i])

In [ ]:
import random
num_to_words = {
    '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
    '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
}

def generate_credit():
    if random.random() > 0.3:
      groups = [''.join(str(random.randint(0, 9)) for _ in range(4)) for _ in range(4)]
    else:
      groups = [' '.join(num_to_words[str(random.randint(0, 9))] for _ in range(4)) for _ in range(4)]
    return '-'.join(groups)

def get_nric_prefix():
    prefixes = ["S", "T", "F", "G"]
    return random.choice(prefixes)

def calculate_nric_suffix(prefix, digits):
    weights = [2, 7, 6, 5, 4, 3, 2]
    sum_val = sum(int(d) * w for d, w in zip(digits, weights))
    if prefix in ["T", "G"]:
        sum_val += 4
    if prefix in ["F", "G"]:
        mapping = "XWUTRQPONMLK"
    else:
        mapping = "JZIHGFEDCBA"

    return mapping[sum_val % 11]

def generate_nric():
    prefix = get_nric_prefix()
    digits = ''.join(str(random.randint(0, 9)) for _ in range(7))
    suffix = calculate_nric_suffix(prefix, digits)
    if random.random() > 0.7:
      digits = " ".join(num_to_words[d] for d in digits)
      return f"{prefix} {digits} {suffix}"
    return f"{prefix}{digits}{suffix}"

def generate_passport():
    if random.random() > 0.7:
      return f"K {' '.join(num_to_words[str(random.randint(0, 9))] for _ in range(7))} {random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}"
    return f"K{''.join(str(random.randint(0, 9)) for _ in range(7))}{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}"

def generate_phone():
    start_digit = random.choice(['6', '8', '9'])
    rest_digits = ''.join(str(random.randint(0, 9)) for _ in range(3))
    second_group = ''.join(str(random.randint(0, 9)) for _ in range(4))
    if random.random() > 0.7:
      start_digit = num_to_words[start_digit]
      rest_digits = " ".join(num_to_words[d] for d in rest_digits)
      second_group = " ".join(num_to_words[d] for d in second_group)
      return f"{start_digit} {rest_digits}-{second_group}"
    return f"{start_digit}{rest_digits}-{second_group}"

def generate_email():
    if random.random() > 0.45:
      user =  ' '.join(generate_person().split()[0]) + ''.join(random.choice('abcdefghijklmnopqrstuvwxyz0123456789') for _ in range(4))
    else:
      user =  (generate_person().split()[0]) + ''.join(random.choice('abcdefghijklmnopqrstuvwxyz0123456789') for _ in range(4))

    domain = random.choice([
    "gmail.com", "mailbox.org", "protonmail.com", "outlook.com", "hotmail.com", "yahoo.com", "icloud.com",
    "aol.com", "zoho.com", "gmx.com", "yandex.com", "mail.com", "fastmail.com", "hushmail.com", "tutanota.com",
    "inbox.com", "me.com", "mac.com", "aim.com", "lycos.com", "rediffmail.com", "juno.com", "mymail.com",
    "runbox.com", "mailfence.com", "posteo.net", "vivaldi.net", "mail.ru", "sbcglobal.net", "bellsouth.net",
    "verizon.net", "earthlink.net", "comcast.net", "cox.net", "charter.net", "frontier.com", "netzero.net",
    "att.net", "zoho.eu", "prodigy.net", "rogers.com", "shaw.ca", "telus.net", "sympatico.ca", "web.de",
    "mail.bg", "123.com", "bigpond.com", "optusnet.com.au", "ozemail.com.au", "internode.on.net",
    "tpg.com.au", "iinet.net.au", "dodo.com.au", "vodafone.com.au", "slingshot.co.nz", "xtra.co.nz",
    "spark.co.nz", "clear.net.nz", "orcon.net.nz", "talktalk.net", "btinternet.com", "virginmedia.com",
    "blueyonder.co.uk", "sky.com", "ntlworld.com", "eircom.net", "o2.ie", "orange.fr", "free.fr", "sfr.fr",
    "laposte.net", "wanadoo.fr", "neuf.fr", "aliceadsl.fr", "libero.it", "virgilio.it", "tiscali.it",
    "tin.it", "tele2.it", "vodafone.it", "fastwebnet.it", "poste.it", "iol.it", "tim.it", "alice.it",
    "netcabo.pt", "clix.pt", "mail.pt", "tvtel.pt", "netmadeira.com", "brisanet.pt", "sapomail.net",
    "orange.es", "telefonica.net", "movistar.es", "terra.es", "wanadoo.es", "ya.com", "jazzfree.com",
    "mundo-r.com", "ono.com", "vodafone.es", "telefónica.es", "euskaltel.com", "optimum.net", "optonline.net"
])
    return f"{user}@{domain}"

def generate_car():
    series = random.choice([chr(i) for i in range(65, 91) if chr(i) not in ["I", "O"]])
    number = ''.join(str(random.randint(0, 9)) for _ in range(4))
    checksum = random.choice([chr(i) for i in range(65, 91) if chr(i) not in ["F", "I", "N", "O", "Q", "V", "W"]])
    return f"S{series}{number}{checksum}"

def generate_bank():
    parts = [
        ''.join(str(random.randint(0, 9)) for _ in range(3)),
        ''.join(str(random.randint(0, 9)) for _ in range(5)),
        str(random.randint(0, 9))
    ]
    if random.random() > 0.7:
      string = '-'.join(parts)
      new_string = ""
      for i in string:
        if i in num_to_words:
            new_string = new_string + num_to_words[i] + " "
      return new_string
    return '-'.join(parts)

def generate_person():
  person_list = [ "aaliyah johnson", "aaron nguyen", "abdul rahman", "abigail cohen", "aditya gupta",
    "aisha brown", "alberto gomez", "alexandra petrova", "ali hassan", "alice zhang",
    "alicia ramirez", "alina schmidt", "amara osei", "amelia thompson", "amir khan",
    "anastasia ivanova", "anders johansson", "andrea rossi", "andres martinez", "angela williams",
    "anika patel", "anna smith", "anwar bakir", "aria kim", "ariana gonzalez",
    "arjun reddy", "asher green", "aya tanaka", "ayesha malik", "aziz youssef",
    "barbara mitchell", "benjamin lee", "binh tran", "boris ivanov", "brandon walker",
    "brenda lopez", "brian o'connor", "camila hernandez", "carlos silva", "carmen reyes",
    "catherine white", "cheng li", "chiara bianchi", "chloe wilson", "christina nielsen",
    "claudia garcia", "colin murphy", "constance mbatha", "cristina neves", "dahlia ahmed",
    "daniel brown", "dariusz lewandowski", "david smith", "deborah green", "deepak kumar",
    "diego rodriguez", "dmitri mikhailov", "dominique dupont", "dung nguyen", "eduardo santos",
    "elena petrova", "elias fischer", "elizabeth johnson", "ella roberts", "emanuel mendes",
    "emilia romero", "emily jones", "emma clark", "enrique ortega", "erin o'brien",
    "esther kim", "ethan wang", "eva nielsen", "fahad hassan", "fatima bint abdullah",
    "fernando garcia", "fiona murray", "francesco ricci", "gabriel rodriguez", "georgia baker",
    "gianna mancini", "grace davis", "gregory adams", "guadalupe flores", "habiba al-mahdi",
    "hana yamamoto", "hans meier", "hassan abdullah", "heather wilson", "hector ramirez",
    "helena ivanova", "henry chen", "hugo rodrigues", "ibrahim khalid", "iggy taylor",
    "imran khan", "inaya sheikh", "ingrid larsen", "irina petrov", "isaac king",
    "isabel garcia", "isabella rossi", "ivan novak", "jackson brown", "jacob cohen",
    "jaime diaz", "jamal williams", "jasmine patel", "jasper lee", "jayden clark",
    "jennifer miller", "jessica wilson", "jia li", "joana mendes", "joel kim",
    "johann schmidt", "john o'connor", "jose luis gonzalez", "josé martínez", "joseph white",
    "josue rivera", "juan perez", "julia weber", "julian meyer", "julie andrews",
    "karen davis", "karim abdullah", "katerina petrova", "katherine lee", "katie brown",
    "kevin mccarthy", "khadija ali", "kiara washington", "kirsten hansen", "kofi boakye",
    "krishna rao", "kumar bhatt", "laila hassan", "larissa almeida", "laura johnson",
    "leah kim", "leandro silva", "leo martin", "leon wu", "leona fox",
    "leslie torres", "liam o'brien", "lila johnson", "lina schmidt", "linda lopez",
    "logan smith", "lorenzo de luca", "lucas brown", "lucia garcia", "ludmila sokolova",
    "luisa fernandez", "lukas novak", "madison clark", "magnus johansson", "mahmoud al-amin",
    "maria fernandez", "mariah jones", "marina volkov", "mario rossi", "markus schneider",
    "marta costa", "martin brown", "maryam khan", "mateo lopez", "matteo ricci",
    "maya patel", "megan turner", "mei li", "melissa johnson", "mia davis",
    "micah anderson", "michael young", "michelle kim", "miguel rodriguez", "mikhail novikov",
    "mina akhtar", "mohammed al-saadi", "mohammad rahman", "molly brown", "monica flores",
    "muhammad ali", "nadia ibrahim", "naomi tanaka", "natalia romanov", "nathaniel smith",
    "neha jain", "nicolas fernandez", "noah walker", "nora schmidt", "olivia taylor",
    "omar gonzalez", "oscar morales", "pablo rodriguez", "paloma garcia", "parker johnson",
    "patricia evans", "patrick murphy", "paul smith", "paula souza", "pedro mendes",
    "peter brown", "priya singh", "qasim ali", "quinn o'neill", "rachel smith",
    "rahul desai", "raj kumar", "ramona lopez", "rashid ahmed", "rebecca williams",
    "renato de souza", "ricardo torres", "roberto marino", "robin taylor", "rosa ramirez",
    "ryan davis", "sabrina khan", "sahil mehta", "samuel moore", "sanjay kumar",
    "sarah lee", "sasha petrov", "savannah johnson", "sebastian garcia", "selena fernandez",
    "serena jones", "shahzad malik", "shelly brown", "sophia white", "sophie harris",
    "stefan popov", "steven wong", "sumit patel", "susana mendez", "syed ali",
    "tahira abdi", "tamara johnson", "tara o'neill", "tariq hasan", "taylor miller",
    "tereza mendes", "terry jones", "thomas white", "thuy nguyen", "tian zhang",
    "tina williams", "timothy brown", "tom johnson", "tshepo nkosi", "usman khan",
    "valeria rossi", "vanessa davis", "vera ivanova", "victor silva", "victoria brown",
    "vincent leclerc", "vivek sharma", "wang wei", "wen li", "william smith",
    "winona johnson", "xavier garcia", "yasmin ali", "yevgeny petrov", "yolanda hernandez",
    "youssef el-khoury", "yuki nakamura", "zainab khan", "zarina ahmed", "zoe clark"
]
  return random.choice(person_list)

def generate_org():
  org_list = [
    "Google", "Microsoft", "Apple", "Amazon", "Facebook", "IBM", "Intel", "Cisco", "Oracle", "HP",
    "Dell", "Samsung", "Sony", "LG", "Panasonic", "Nokia", "Siemens", "Tesla", "SpaceX", "Alibaba",
    "Tencent", "Baidu", "Huawei", "Xiaomi", "PayPal", "Adobe", "Salesforce", "Netflix", "Spotify",
    "Uber", "Airbnb", "Lyft", "Shopify", "Pinterest", "LinkedIn", "Snapchat", "Twitter", "WeWork",
    "Stripe", "Square", "Dropbox", "Slack", "Zoom", "Reddit", "TikTok", "Twitch", "Snap", "Qualcomm",
    "Nvidia", "AMD", "Accenture", "Adobe Systems", "Amazon Web Services", "American Express", "AOL",
    "AT&T", "Autodesk", "Boeing", "Broadcom", "Capgemini", "CGI", "Cognizant", "Comcast", "CrowdStrike",
    "Dell Technologies", "eBay", "Electronic Arts", "Ericsson", "Expedia", "Ford Motor Company", "Fujitsu",
    "Garmin", "General Electric", "General Motors", "GitHub", "GoDaddy", "Hewlett Packard Enterprise",
    "Hitachi", "Honeywell", "Hyundai", "Infosys", "Intuit", "J.P. Morgan", "JD.com", "Juniper Networks",
    "Lenovo", "Logitech", "Mastercard", "McAfee", "Micron Technology", "Nasdaq", "NCR Corporation", "Nokia Corporation",
    "Northrop Grumman", "NTT Data", "Okta", "Palantir", "Palo Alto Networks", "Paytm", "Qualys", "Rakuten",
    "Raytheon Technologies", "SAP", "Seagate Technology", "ServiceNow", "Siemens AG", "SoftBank Group", "Sony Corporation",
    "Square Inc.", "Symantec", "Synopsys", "Tech Mahindra", "Texas Instruments", "Thales Group", "T-Mobile",
    "Toshiba", "Toyota", "Uber Technologies", "Verizon", "Visa", "VMware", "Western Digital", "Wipro",
    "Workday", "Xerox", "Yahoo", "Yandex", "Zillow", "ZTE", "Zoom Video Communications", "Zscaler",
    "Samsung Electronics", "LG Electronics", "Foxconn", "Tencent Holdings", "Alibaba Group", "Baidu Inc.",
    "JD.com Inc.", "NetEase", "Sina Corporation", "Weibo Corporation", "Xiaomi Corporation", "Meituan Dianping",
    "Didi Chuxing", "ByteDance", "Pinduoduo", "Sogou", "58.com", "China Mobile", "China Telecom", "China Unicom"
]

  return random.choice(org_list)

def generate_cardinal():
  cardinal_list = [
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen",
    "seventeen", "eighteen", "nineteen", "twenty", "twenty-one", "twenty-two", "twenty-three", "twenty-four", "twenty-five", "twenty-six", "twenty-seven",
    "twenty-eight", "twenty-nine", "thirty", "thirty-one", "thirty-two", "thirty-three", "thirty-four", "thirty-five", "thirty-six", "thirty-seven",
    "thirty-eight", "thirty-nine", "forty", "forty-one", "forty-two", "forty-three", "forty-four", "forty-five", "forty-six", "forty-seven", "forty-eight",
    "forty-nine", "fifty"
]

  return random.choice(cardinal_list)

def generate_gpe():
  gpe_list = [
      "singapore", "malaysia", "indonesia", "philippines", "vietnam", "laos", "cambodia", "myanmar", "brunei", "east_timor",
      "thailand", "india", "china", "japan", "south_korea", "north_korea", "taiwan", "hong_kong", "macau", "mongolia",
      "australia", "new_zealand", "papua_new_guinea", "fiji", "tonga", "samoa", "vanuatu", "solomon_islands", "palau", "micronesia",
      "nauru", "kiribati", "marshall_islands", "tuvalu", "maldives", "sri_lanka", "bangladesh", "nepal", "bhutan", "pakistan",
      "afghanistan", "iran", "iraq", "syria", "lebanon", "jordan", "israel", "saudi_arabia", "uae", "qatar", "kuwait",
      "bahrain", "united_states", "canada", "mexico", "brazil", "argentina", "chile", "peru", "colombia", "venezuela",
      "uruguay", "paraguay", "bolivia", "ecuador", "guyana", "suriname", "france", "germany", "united_kingdom", "italy",
      "spain", "portugal", "netherlands", "belgium", "switzerland", "austria", "sweden", "norway", "denmark", "finland",
      "iceland", "ireland", "greece", "turkey", "russia", "ukraine", "belarus", "poland", "czech_republic", "slovakia",
      "hungary", "romania", "bulgaria", "serbia", "croatia", "slovenia", "bosnia_and_herzegovina", "montenegro", "macedonia",
      "albania", "kosovo", "luxembourg", "monaco", "andorra", "liechtenstein", "san_marino", "vatican_city", "morocco",
      "algeria", "tunisia", "libya", "egypt", "sudan", "south_sudan", "ethiopia", "somalia", "kenya", "uganda",
      "rwanda", "burundi", "tanzania", "zambia", "zimbabwe", "mozambique", "botswana", "namibia", "south_africa", "lesotho",
      "swaziland", "angola", "drc", "congo", "gabon", "equatorial_guinea", "cameroon", "nigeria", "niger", "mali",
      "chad", "central_african_republic", "sudan", "eritrea", "djibouti", "madagascar", "comoros", "seychelles", "mauritius",
      "mauritania", "senegal", "gambia", "guinea", "guinea_bissau", "sierra_leone", "liberia", "cote_d'ivoire", "ghana",
      "togo", "benin", "burkina_faso", "mali", "greenland", "iceland", "barbados", "bahamas", "cuba", "jamaica",
      "haiti", "dominican_republic", "puerto_rico", "trinidad_and_tobago", "saint_lucia", "saint_vincent_and_the_grenadines", "grenada",
      "saint_kitts_and_nevis", "antigua_and_barbuda", "dominica", "brazil", "colombia", "argentina", "chile", "peru",
      "venezuela", "uruguay", "paraguay", "bolivia", "ecuador", "guyana", "suriname", "french_guiana", "falkland_islands",
      "greenland", "iceland", "barbados", "bahamas", "cuba", "jamaica", "haiti", "dominican_republic", "puerto_rico",
      "trinidad_and_tobago", "saint_lucia", "saint_vincent_and_the_grenadines", "grenada", "saint_kitts_and_nevis", "antigua_and_barbuda",
      "dominica", "antarctica", "arctic", "the_amazon", "sahara_desert", "mount_everest", "great_barrier_reef", "niagara_falls",
      "grand_canyon", "victoria_falls", "serengeti_national_park", "kruger_national_park", "banff_national_park", "yosemite_national_park",
      "yellowstone_national_park", "galapagos_islands", "machu_picchu", "angkor_wat", "the_great_wall_of_china", "petra",
      "taj_mahal", "stonehenge", "eiffel_tower", "colosseum", "sydney_opera_house", "burj_khalifa"
  ]
  return random.choice(gpe_list)

def generate_time():
  time_list = [
    "one day one night", "two days one night", "three days two nights", "four days three nights", "five days four nights", "six days five nights", "seven days six nights",
    "eight days seven nights", "nine days eight nights", "ten days nine nights", "eleven days ten nights", "twelve days eleven nights", "thirteen days twelve nights",
    "fourteen days thirteen nights", "fifteen days fourteen nights", "sixteen days fifteen nights", "seventeen days sixteen nights", "eighteen days seventeen nights",
    "nineteen days eighteen nights", "twenty days nineteen nights", "twenty-one days twenty nights", "twenty-two days twenty-one nights", "twenty-three days twenty-two nights",
    "twenty-four days twenty-three nights", "twenty-five days twenty-four nights", "twenty-six days twenty-five nights", "twenty-seven days twenty-six nights",
    "twenty-eight days twenty-seven nights", "twenty-nine days twenty-eight nights", "thirty days twenty-nine nights", "thirty-one days thirty nights",
    "thirty-two days thirty-one nights", "thirty-three days thirty-two nights", "thirty-four days thirty-three nights", "thirty-five days thirty-four nights",
    "thirty-six days thirty-five nights", "thirty-seven days thirty-six nights", "thirty-eight days thirty-seven nights", "thirty-nine days thirty-eight nights",
    "forty days thirty-nine nights", "forty-one days forty nights", "forty-two days forty-one nights", "forty-three days forty-two nights", "forty-four days forty-three nights",
    "forty-five days forty-four nights", "forty-six days forty-five nights", "forty-seven days forty-six nights", "forty-eight days forty-seven nights", "forty-nine days forty-eight nights",
    "one hour", "two hours", "three hours", "four hours", "five hours", "six hours", "seven hours", "eight hours", "nine hours", "ten hours",
    "eleven hours", "twelve hours", "thirteen hours", "fourteen hours", "fifteen hours", "sixteen hours", "seventeen hours", "eighteen hours", "nineteen hours", "twenty hours",
    "twenty-one hours", "twenty-two hours", "twenty-three hours", "twenty-four hours", "one minute", "two minutes", "three minutes", "four minutes", "five minutes",
    "ten minutes", "fifteen minutes", "twenty minutes", "thirty minutes", "one second", "two seconds", "three seconds", "four seconds", "five seconds"
]
  return random.choice(time_list)

def generate_date():
  date_list =[
    "first of january", "second of february", "third of march", "fourth of april", "fifth of may", "sixth of june",
    "seventh of july", "eighth of august", "ninth of september", "tenth of october", "eleventh of november",
    "twelfth of december", "thirteenth of january", "fourteenth of february", "fifteenth of march", "sixteenth of april",
    "seventeenth of may", "eighteenth of june", "nineteenth of july", "twentieth of august", "twenty-first of september",
    "twenty-second of october", "twenty-third of november", "twenty-fourth of december", "twenty-fifth of january",
    "twenty-sixth of february", "twenty-seventh of march", "twenty-eighth of april", "twenty-ninth of may",
    "thirtieth of june", "thirty-first of july", "first of august", "second of september", "third of october",
    "fourth of november", "fifth of december", "sixth of january", "seventh of february", "eighth of march",
    "ninth of april", "tenth of may", "eleventh of june", "twelfth of july", "thirteenth of august", "fourteenth of september",
    "fifteenth of october", "sixteenth of november", "seventeenth of december", "eighteenth of january", "nineteenth of february",
    "1 january", "2 february", "3 march", "4 april", "5 may", "6 june", "7 july", "8 august", "9 september", "10 october",
    "11 november", "12 december", "13 january", "14 february", "15 march", "16 april", "17 may", "18 june", "19 july",
    "20 august", "21 september", "22 october", "23 november", "24 december", "25 january", "26 february", "27 march",
    "28 april", "29 may", "30 june", "31 july", "1 august", "2 september", "3 october", "4 november", "5 december",
    "6 january", "7 february", "8 march", "9 april", "10 may", "11 june", "12 july", "13 august", "14 september",
    "15 october", "16 november", "17 december", "18 january", "19 february", "5 july",
    "tomorrow", "day after tomorrow", "next week", "next month", "next year", "yesterday", "day before yesterday", "last week", "last month", "last year",
    "new year's day", "valentine's day", "st. patrick's day", "easter sunday", "good friday", "halloween", "thanksgiving",
    "christmas", "independence day", "labor day", "memorial day", "martin luther king jr. day", "presidents' day",
    "veterans day", "columbus day", "juneteenth", "hanukkah", "ramadan", "diwali", "chinese new year", "mardi gras",
    "earth day", "april fools' day", "cinco de mayo", "father's day", "mother's day", "international women's day",
    "flag day", "patriot day", "national dog day", "black friday", "cyber monday"
]


  return random.choice(date_list)

tagss = []
for i in range(len(seed_data[1])):
  for j in seed_data[1][i]:
    if j['label'] not in tagss:
      tagss.append(j['label'])

funcss = [generate_nric, generate_phone, generate_person, generate_org, generate_bank, generate_cardinal, generate_gpe, generate_car, generate_time, generate_email, generate_passport, generate_credit, generate_date]
generate_dict = dict(zip(tagss, funcss))

def data_to_prompt(data, tags):
  prompt = ""
  prompt = prompt + data + ". NER Tags = ("
  for tag in tags:
    prompt += data[tag['start']:tag['end']] + "-" + tag['label']
    prompt += ", "
  prompt = prompt + ")"
  return prompt

In [ ]:
manual_lookup = []
replaced_data = []

for i in range(len(augmented_data)):

  data = augmented_data[i].split('\n')
  tags = seed_data[1][i]

  for sentence in data:
    sentence = re.sub(r'[",.]', "", sentence)
    sentence = re.sub(r"[']", "", sentence)
    sentence = sentence.strip()
    sentence = sentence[2:]
    sentence = sentence.strip()

    tag_list = {}
    flag = True
    for j in tags:

      original_tag = seed_data[0][i][j['start']:j["end"]]
      label = j['label']
      new_tag = generate_dict[label]()

      if sentence.find(original_tag) != -1:
        new_tag_x = " "+new_tag+" "
        sentence = sentence.replace(original_tag, new_tag_x)
        tag_list[label] = new_tag

      else:
        flag = False
        manual_lookup.append(sentence)

    if flag:
      sentence = sentence.replace("  ", " ")
      replaced_data.append([sentence, tag_list])

In [ ]:
def find_word_indices(sentence, word):
    start_index = sentence.find(word)
    if start_index == -1:
        return None, None
    end_index = start_index + len(word)
    return start_index, end_index

In [ ]:
aug2 = [[], []]
for i in replaced_data:
  aug2[0].append(i[0])
  temp = []
  for j in i[1]:
    start, end = find_word_indices(i[0], i[1][j])
    if start is not None and end is not None:
      temp.append({"start": start, "end": end, "label": j})
  aug2[1].append(temp)

In [ ]:
def sublist_indices(big, small):
  start = small[0]
  for i in range(len(big)):
    if big[i] == start:
      if big[i+len(small)-1] == small[-1]:
        return [i, i+len(small)-1]
  return -1

In [ ]:
# Adding noise
speech_noises = [
    "(uh)", "(um)", "(er)", "(ah)", "(eh)", "(hmm)", "(mm-hmm)", "(uh-huh)", "(uh-uh)", "(oh)",
    "(huh)", "(yeah)", "(like)", "(you know)", "(right)", "(so)", "(actually)", "(basically)",
    "(literally)", "(I mean)", "(well)", "(kind of)", "(sort of)", "(uhm)", "(ohh)", "(ahh)",
    "(umm)", "(err)", "(mmm)", "(tsk)", "(ahem)", "(gosh)", "(whoa)", "(wow)", "(oops)", "(uh-oh)",
    "(phew)", "(eek)", "(shh)", "(huh-uh)"
]

texts = []
for i in range(len(aug2[0])):

  if random.random() > 0.0001:
    sent_split = aug2[0][i].split(" ")
    indices = []
    indices = random.sample(range(0, len(sent_split)), 1)

    for ind in indices:
      tag_shift = []
      for tag in range(len(aug2[1][i])):
        try:
          val = sublist_indices(sent_split, replaced_data[i][1][list(replaced_data[i][1].keys())[tag]].strip().split(" "))
        except:
          val = -1
        if val != -1:
          if val[0] == val[1]:
            if ind <= val[0]:
              tag_shift.append(1)
            else:
              tag_shift.append(0)
          else:
            if ind <= val[0]:
              tag_shift.append(1)
            elif ind > val[1]:
              tag_shift.append(0)
            elif ind > val[0] and ind <= val[1]:
              tag_shift.append(2)
        else:
          tag_shift.append(-1)

      if -1 not in tag_shift:
        noise = random.sample(speech_noises, 1)[0]
        sent_split.insert(ind, noise)
        for j in range(len(tag_shift)):
          if tag_shift[j] == 1:
            aug2[1][i][j]['start'] += len(noise)+1
            aug2[1][i][j]['end'] += len(noise)+1
          elif tag_shift[j] == 2:
            aug2[1][i][j]['end'] += len(noise)+1
          elif tag_shift[j] == 0:
            pass
    texts.append(" ".join(sent_split))
  else:
    texts.append(aug2[0][i])

In [ ]:
for i in range(len(texts)):
  for j in range(len(aug2[1][i])):
    start = aug2[1][i][j]['start']
    end = aug2[1][i][j]['end']
    label = aug2[1][i][j]['label']
    if texts[i][end-1] == " ":
      aug2[1][i][j]['end'] = aug2[1][i][j]['end']- 1

In [ ]:
aug2[0] = texts

In [ ]:
for i in range(len(aug[0])):
  if aug[1][i] == []:
    aug2[0].append(aug[0][i])
    aug2[1].append(aug[1][i])

In [ ]:
combined = list(zip(aug2[0], aug2[1]))
random.shuffle(combined)

aug2[0], aug2[1] = zip(*combined)
aug2[0] = list(aug2[0])
aug2[1] = list(aug2[1])

In [ ]:
import pickle
with open('diverse.pkl', 'wb') as f:
  pickle.dump(aug2, f)